# Part A3 - Evolution

## Setup

In [3]:
from py2neo import Graph, ClientError

In [4]:
#set connection variables
PORT = "7687" #database running on this port for bolt connections
USER = "neo4j" #standard user
PASSWORD = "publication-graph" #db password

In [5]:
#connect to database
try:
    graph = Graph('bolt://localhost:'+PORT, auth=(USER, PASSWORD))
    print('SUCCESS: Connected to the Neo4j Database.')
except Exception as e:
    print('ERROR: Could not connect to the Neo4j Database. See console for details.')
    raise SystemExit(e)

SUCCESS: Connected to the Neo4j Database.


In [6]:
# query helper function
def run_query(query:str):
    try:
        return graph.run(query)
    except ClientError as e:
        print(e.message)

## Reviews

In [6]:
REVIEW_CONTENT_QUERY = """
//reviewers
Load csv with headers from
'file:///reviewers.csv' AS line

FIELDTERMINATOR ','

    call{
      with line
      Match (paper:Paper {paper_key: line.`key:string`})<-[:REVIEW_SUBJECT]-(review:Review)
      Match (r:Researcher {name: line.reviewers})-[pi:PARTICIPATES_IN]->(review)

      Set pi.suggested_decision = line.suggested_decision,
          pi.review_content = line.review_text,
          pi.supports_acceptance = toBoolean(line.supports_acceptance)

    }in transactions

"""

In [7]:
run_query(REVIEW_CONTENT_QUERY)

(No data)

## Affiliations

In [2]:
AFFILIATION = """
Load csv with headers from
  'file:///affiliations.csv' AS line

    FIELDTERMINATOR ','

call{
  with line
  
  Match (r:Researcher {name: line.author_list})
  Merge (o:Organization {organization_name: line.affiliation})
  set o.organization_type = line.organization_type

  Merge (r)-[:AFFILIATED_WITH]->(o)
  
}in transactions
"""

In [7]:
run_query(AFFILIATION)

(No data)